In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model


2022-08-03 19:29:29.567787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-03 19:29:29.568016: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [26]:
class Convolutional_Autoencoder_gamma_e(Model):
    def __init__(self, latent_dim):
        super(Convolutional_Autoencoder_gamma_e, self).__init__()

        self.encoder = tf.keras.Sequential([
          layers.Input(shape=(40, 40, 1)),
          layers.Conv2D(10, kernel_size=4, strides=1, activation='relu', padding='same'),
          layers.Conv2D(10, kernel_size=4, strides=1, activation='relu', padding='same'),
          layers.AveragePooling2D(pool_size=2),
          layers.Conv2D(5, kernel_size=4, strides=1, activation='relu', padding='same'),
          layers.Conv2D(5, kernel_size=4, strides=1, activation='relu', padding='same'),#,
          layers.Flatten(),
          layers.Dense(400, activation='relu'),
          layers.Dense(100, activation='relu'),
          layers.Dense(latent_dim, activation='sigmoid')])

        self.decoder = tf.keras.Sequential([
          layers.Dense(100, activation='relu'),
          layers.Dense(400, activation='relu'),
          layers.Reshape((20, 20, 1)),
          layers.Conv2D(5, kernel_size=4, strides=1, activation='relu', padding='same'),# after 14
          layers.Conv2D(5, kernel_size=4, strides=1, activation='relu', padding='same'),
          layers.UpSampling2D(size=2),
          layers.Conv2D(10, kernel_size=4, strides=1, activation='relu', padding='same'),# after 14
          layers.Conv2D(1, kernel_size=4, strides=1, activation='relu', padding='same')])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [27]:
a = Convolutional_Autoencoder_gamma_e(32)

In [28]:
a.build(input_shape=(1,40,40,1))

In [29]:
a.summary()

Model: "convolutional__autoencoder_gamma_e_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_6 (Sequential)   (None, 32)                846722    
                                                                 
 sequential_7 (Sequential)   (1, 40, 40, 1)            45161     
                                                                 
Total params: 891,883
Trainable params: 891,883
Non-trainable params: 0
_________________________________________________________________


In [30]:
a.encoder.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 40, 40, 10)        170       
                                                                 
 conv2d_23 (Conv2D)          (None, 40, 40, 10)        1610      
                                                                 
 average_pooling2d_1 (Averag  (None, 20, 20, 10)       0         
 ePooling2D)                                                     
                                                                 
 conv2d_24 (Conv2D)          (None, 20, 20, 5)         805       
                                                                 
 conv2d_25 (Conv2D)          (None, 20, 20, 5)         405       
                                                                 
 flatten_4 (Flatten)         (None, 2000)              0         
                                                      

In [25]:
a.decoder.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (1, 100)                  3300      
                                                                 
 dense_8 (Dense)             (1, 400)                  40400     
                                                                 
 reshape_2 (Reshape)         (1, 20, 20, 1)            0         
                                                                 
 conv2d_18 (Conv2D)          (1, 20, 20, 5)            85        
                                                                 
 conv2d_19 (Conv2D)          (1, 20, 20, 5)            405       
                                                                 
 up_sampling2d (UpSampling2D  (1, 40, 40, 5)           0         
 )                                                               
                                                      